$\DeclareMathOperator{\opdiv}{div}$

The Hellan Herrmann Johnson Method for Kirchhoff plates
===


Genuine, but not well known method for $4^{th}$ order problems

Hellan 67, Herrmann 67, Johnson 73, Arnold+Brezzi 85, Comodi 89,
Krendl+Rafetseder+Zulehner 16, Chen+Hu+Huang 16, Braess+Pechstein+JS 17

Is an arbitrary order ($k \geq 0$) mixed discretization method on
possibly curved, mapped triangular elements.

Allows hybridization to obtain a positive definite system matrix,
lowest order case corresponds to the Morley element.

The (newer) TD-NNS method is tightly connected to the (much older) HHJ method.

With Braess: Prager-Synge type constant-free a posteriori error
estimates using $C^0-DG$ and HHJ

Kirchhoff Plate equation
--


Find vertical deflection $w$ such that

$$
\opdiv \opdiv \nabla^2 w = f \qquad \text{ + boundary conditions}
$$

Reduce to second order system:

\begin{eqnarray*}
\sigma - \nabla^2 w & = & 0 \\
\opdiv \opdiv \sigma & = & f
\end{eqnarray*}

Variational formulation with $\sigma \in H(\opdiv)^{SYM}$ and $w \in H^1$:

$$
\begin{array}{ccccll}
\int \sigma \tau & + & \int \opdiv \tau \, \nabla w & = & 0 & \forall \, \tau \\[0.5em]
\int \opdiv \sigma \nabla v & & & = & \int f v & \forall \, v
\end{array}
$$

Hellan-Herrmann-Johnson method
---

Find $\sigma_h \in \Sigma_h \subset H(\opdiv \opdiv)$ and $w_h \in W_h \subset H^1$ such that

$$
\begin{array}{ccccll}
\int \sigma_h \tau & + & \sum_T \int_T \opdiv \tau \, \nabla w_h + \int_{\partial T} \tau_{nt} \nabla_t w_h & = & 0 & \forall \, \tau \\[0.5em]
\sum_T \int_T \opdiv \sigma_h \nabla v + \int_{\partial_T} \sigma_{nt} \nabla_t v & & & = & \int f v & \forall \, v
\end{array}
$$

This mixed method satisfies the magic discrete kernel inclusion

$$
V_{h,0} \subset V_0
$$

leading to the best-approximation property of the bending moments $\sigma$:

$$
\| \sigma - \sigma_h \|_{L_2} \leq \inf_{\tau_h \in \Sigma_h} \| \sigma - \tau_h \|_{L_2}  + \| f - I_h f \|
$$


In [1]:
from ngsolve import *
from netgen.geom2d import unit_square
import netgen.gui

In [2]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.1))
Draw (mesh)

order = 2
Sigma = HDivDiv(mesh, order=order)
W = H1(mesh, order=order+1, dirichlet=[1,2,3,4])
X = FESpace([Sigma,W])

print ("ndof-Sigma:", Sigma.ndof, ", ndof-W:", W.ndof, ", ndof-X:", X.ndof)

ndof-Sigma: 3165 , ndof-W: 1096 , ndof-X: 4261


In [3]:
sigma, u = X.TrialFunction()
tau, v = X.TestFunction()

n = specialcf.normal(2)
def tang(u): return u-(u*n)*n

a = BilinearForm(X, symmetric=True)
a += SymbolicBFI (InnerProduct (sigma, tau) + div(sigma)*grad(v) 
                  + div(tau)*grad(u))
a += SymbolicBFI (-(sigma*n) * tang(grad(v)) - (tau*n)*tang(grad(u)), 
                  element_boundary=True)
a.Assemble()

f = LinearForm(X)
f += SymbolicLFI (500 * v)
f.Assemble()

u = GridFunction(X)
u.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec

Draw (u.components[0], mesh, name="sigma")
Draw (u.components[1], mesh, name="disp")